<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Preprocessing/Text%20Preprocessing%20layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

Working with preprocessing layers

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
data = ["How", "are", "you?"]

In [3]:
layer = layers.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

tf.Tensor(
[[3]
 [4]
 [2]], shape=(3, 1), dtype=int64)
